In [1]:
# Following https://github.com/huggingface/notebooks/blob/master/examples/translation.ipynb

In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/ 

In [3]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [4]:
!unzip lc-quad-wikidata-2021-07-09.zip

Archive:  lc-quad-wikidata-2021-07-09.zip
 extracting: lc-quad-wikidata-2021-07-09/dataset_dict.json  
   creating: lc-quad-wikidata-2021-07-09/test/
  inflating: lc-quad-wikidata-2021-07-09/test/dataset.arrow  
  inflating: lc-quad-wikidata-2021-07-09/test/dataset_info.json  
  inflating: lc-quad-wikidata-2021-07-09/test/state.json  
   creating: lc-quad-wikidata-2021-07-09/train/
  inflating: lc-quad-wikidata-2021-07-09/train/dataset.arrow  
  inflating: lc-quad-wikidata-2021-07-09/train/dataset_info.json  
  inflating: lc-quad-wikidata-2021-07-09/train/state.json  


In [5]:
!nvidia-smi

Mon Aug  9 10:34:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    33W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
model_checkpoint = 't5-base'
gdir = 'drive/My Drive/Colab Notebooks/t5-2021-07-11/'
model_name='sparql-translator-2021-08-09-'+model_checkpoint
model_path='./models/'+model_name
ds_path= 'lc-quad-wikidata-2021-07-09'

In [7]:
!mkdir models

In [8]:
!pip install datasets transformers sacrebleu -qqq

     |████████████████████████████████| 264 kB 4.1 MB/s 
     |████████████████████████████████| 2.6 MB 7.4 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 76 kB 4.2 MB/s 
     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 118 kB 28.3 MB/s 
     |████████████████████████████████| 243 kB 30.2 MB/s 
     |████████████████████████████████| 895 kB 30.8 MB/s 
     |████████████████████████████████| 636 kB 39.7 MB/s 
     |████████████████████████████████| 3.3 MB 44.5 MB/s 


In [9]:
from datasets import load_dataset, load_metric, Dataset, load_from_disk
raw_datasets = load_from_disk(ds_path)

In [10]:
!pip install tqdm==4.49.0

     |████████████████████████████████| 69 kB 3.0 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.0
    Uninstalling tqdm-4.62.0:
      Successfully uninstalled tqdm-4.62.0


In [11]:
from tqdm import tqdm

In [12]:
metric = load_metric("sacrebleu")

Downloading:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

In [13]:
# Preprocessing

In [14]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [15]:
print(raw_datasets['test']['translation'][0]['sparql'])

select distinct ?sbj where [ ?sbj wdt:head_of_state wd:mahmoud_abbas . ?sbj wdt:instance_of wd:country ]


In [16]:
tokenizer.decode(tokenizer(raw_datasets['test']['translation'][0]['sparql'])['input_ids'])

'select distinct?sbj where [?sbj wdt:head_of_state wd:mahmoud_abbas.?sbj wdt:instance_of wd:country ]</s>'

In [17]:
print(raw_datasets['test']['translation'][0]['en'],tokenizer(raw_datasets['test']['translation'][0]['en']))

Who is the  {country} for {head of state} of {Mahmoud Abbas} {'input_ids': [2645, 19, 8, 3, 2, 17529, 2, 21, 3, 2, 3313, 13, 538, 2, 13, 3, 2, 329, 9, 107, 51, 1063, 26, 891, 4883, 2, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [18]:
prefix = "translate English to Sparql: "

In [19]:
max_input_length = 0 
max_target_length = 0
for d in tqdm(raw_datasets['train']['translation']):
    len_en = len(d['en'])
    len_qry = len(d['sparql'])
    if len_en > max_input_length: max_input_length=len_en
    if len_qry > max_target_length: max_target_length=len_qry

100%|██████████| 38175/38175 [00:00<00:00, 1115755.13it/s]


In [20]:
print(max_input_length, max_target_length)

248 349


In [21]:
source_lang = "en"
target_lang = "sparql"

def preprocess_function(examples):
    inputs = []
    targets= []
    for ex in examples["translation"]:
      inputs.append(prefix + ex[source_lang])
      targets.append(ex[target_lang])
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
      labels = tokenizer(targets, max_length=max_target_length, truncation=True)
      
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [22]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/39 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [23]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 38175
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'labels', 'translation'],
        num_rows: 9485
    })
})

In [24]:
# Fine-tuning the model

In [25]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [26]:
batch_size = 8
args = Seq2SeqTrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
)

In [27]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [28]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'bp': 1.0,
 'counts': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'ref_len': 4,
 'score': 0.0,
 'sys_len': 4,
 'totals': [4, 2, 0, 0]}

In [29]:
# The last thing to define for our Seq2SeqTrainer is how to compute 
# the metrics from the predictions. We need to define 
# a function for this, which will just use the metric we loaded earlier, 
# and we have to do a bit of pre-processing to decode the predictions into texts:

import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.replace('?',' ?').replace('.', ' .').strip() for pred in preds]
    labels = [[label.replace('?',' ?').replace('.', ' .').strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [30]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp fp16 backend


In [ ]:
%%time
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: translation.
***** Running training *****
  Num examples = 38175
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4772


Epoch,Training Loss,Validation Loss


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1310: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
Saving model checkpoint to sparql-translator-2021-08-09-t5-base/checkpoint-500
Configuration saved in sparql-translator-2021-08-09-t5-base/checkpoint-500/config.json
Model weights saved in sparql-translator-2021-08-09-t5-base/checkpoint-500/pytorch_model.bin
tokenizer config file saved in sparql-translator-2021-08-09-t5-base/checkpoint-500/tokenizer_config.json
Special tokens file saved in sparql-translator-2021-08-09-t5-base/checkpoint-500/special_tokens_map.json
Saving model checkpoint to sparql-translator-2021-08-09-t5-base/checkpoint-1000
Configuration saved in sparql-translator-2021-08-09

In [ ]:
trainer.save_model(model_path)

In [ ]:
!ls -l --block-size=M {model_path}

In [ ]:
!mkdir drive/MyDrive/{model_path}

In [ ]:
!cp {model_path}/* drive/MyDrive/{model_path}/